<a href="https://colab.research.google.com/github/RileyHutch/deep-learning-challenge/blob/main/Optimized_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# **Preprocess the data based off work done in Starter_Code.ipynb**

In [31]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [32]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [33]:
# Look at ASK_AMT value counts to identify and replace with "Other"               ASK_AMT has 8,747 unique values which is significantly more than the next closest. Looking to simplify this.
amt_counts = application_df['ASK_AMT'].value_counts()

In [34]:
# Try to find a good cutoff value for ASK_AMT
filtered_amt_counts = amt_counts[amt_counts > 1]
filtered_amt_counts


,count
ASK_AMT,
5000,25398
10478,3
15583,3
63981,3
6725,3
...,...
100000,2
8181,2
24746,2


In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
ASK_AMT_to_replace = list(amt_counts[amt_counts < 3].index)

# Replace in dataframe
for unit in ASK_AMT_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(unit,"Other")

# Check to make sure replacement was successful
application_df['ASK_AMT'].value_counts()

,count
ASK_AMT,
5000,25398
Other,8889
15583,3
6725,3
10478,3
63981,3


In [36]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [37]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [38]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
counts = application_df['CLASSIFICATION'].value_counts()
counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [39]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_counts = counts[counts > 1]
filtered_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [40]:
# Choose a cutoff value and create a list of classifications to be replaced

# use the variable name `classifications_to_replace`
classifications_to_replace = list(counts[counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [41]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_6725,ASK_AMT_10478,ASK_AMT_15583,ASK_AMT_63981,ASK_AMT_Other
0,1,1,False,True,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False
1,1,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,1,0,False,False,False,False,False,True,False,False,...,False,False,True,False,True,False,False,False,False,False
3,1,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4,1,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [42]:
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
X=application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **Compile, Train and Evaluate the Model**

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Adding 2 additinal hidden layers as well as increasing the number of neurons in each layer. All activation funcitons are relu.

# First hidden layer   relu is commonly used in deep networks
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=48))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer          output needs to be sigmoid becasue we are trying to classify as yes/no (binary)
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 100)                 │           4,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 80)                  │           8,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 60)                  │           4,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 40)                  │           2,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              41 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,321 (79.38 KB)

 Trainable params: 20,321 (79.38 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
print(X_train_scaled.shape)


(25724, 48)


In [47]:
# Train the model       +20 epochs from starter_code
fit_model = nn.fit(X_train_scaled, y_train, epochs=120)

Epoch 1/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7065 - loss: 0.5910
Epoch 2/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7308 - loss: 0.5527
Epoch 3/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7314 - loss: 0.5484
Epoch 4/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7321 - loss: 0.5500
Epoch 5/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7309 - loss: 0.5478
Epoch 6/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7337 - loss: 0.5480
Epoch 7/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7348 - loss: 0.5456
Epoch 8/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7373 - loss: 0.5439
Epoch 9/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7332 - loss: 0.5450
Epoch 10/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7377 - loss: 0.5413
Epoch 11/120
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7322 - loss: 0.5445
Epoch 12/120
804/804 ━━━━━━━━━━━━━━━━━━━━

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7282 - loss: 0.5986
Loss: 0.598634660243988, Accuracy: 0.7281632423400879


In [49]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")